In [2]:
import findspark
findspark.init('/opt/cloudera/parcels/SPARK2-2.3.0.cloudera3-1.cdh5.13.3.p0.458809/lib/spark2')
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('victor_joseph_jhonatan').getOrCreate()

In [3]:
spark.sql("show tables from datag3").show()

+--------+--------------------+-----------+
|database|           tableName|isTemporary|
+--------+--------------------+-----------+
|  datag3|       base_clientes|      false|
|  datag3|data_in_base_clie...|      false|
|  datag3|data_in_info_clie...|      false|
|  datag3|       info_clientes|      false|
+--------+--------------------+-----------+



In [4]:
df_base_clientes = spark.sql("select * from datag3.data_in_base_clientes")
df_base_clientes.show()

+---------------+----------+-------------------+-----------------+----+----+------------+---------+-------------+------------------+-----------+
|NUMERODOCUMENTO|    APEPAT|             APEMAT|          NOMBRES|EDAD|SEXO|DEPARTAMENTO|PROVINCIA|     DISTRITO|ESTADOCIVILTITULAR|IncomeLevel|
+---------------+----------+-------------------+-----------------+----+----+------------+---------+-------------+------------------+-----------+
|       41854018|     PINTO|            CONDORI|        JOSE LUIS|  31|   M|        LIMA|     LIMA|         LIMA|                 S|          1|
|       41020654|    TORRES|               NAPA|     LUIS ENRIQUE|  34|   M|        LIMA|     LIMA|         LIMA|                 S|          1|
|       41591162| CASTANEDA|          TANTALEAN|  SHIRLEY JANETTE|  33|   F|        LIMA|     LIMA|         LIMA|                 S|          1|
|       41034300|    TAPARA|             FLORES| MARIELA MERCEDES|  37|   F|        LIMA|     LIMA|         LIMA|                 

In [5]:
df_info_clientes = spark.sql("select * from datag3.data_in_info_clientes")
df_info_clientes.show()

+----------+---+-------+----------+--------+-----------------+------+----------+
|CustomerId|TOB|  Bal01|MaxDqBin01|MtgBal01|NonBankTradesDq01|FlagGB|FlagSample|
+----------+---+-------+----------+--------+-----------------+------+----------+
|  41854018| 44| 604.86|         0|       0|                0|     1|         1|
|  41020654| 79|1006.21|         0|       0|                0|     1|         1|
|  41591162|102| 299.23|         0|       0|                0|     1|         1|
|  41034300|  0| 645.19|         0|       0|                0|     1|         1|
|  41150271|109|    218|         0|       0|                0|     1|         1|
|  41125546|183|  10.32|         0|       0|                0|     1|         1|
|  41919700|172|3517.07|         0|       0|                0|     1|         1|
|  41678951| 76| 216.52|         0|       0|                0|     1|         1|
|  41676527|136|1908.69|         0|       0|                0|     1|         1|
|  41062987|171|1895.19|    

In [6]:
DF3 = df_base_clientes.join(df_info_clientes, [df_base_clientes.NUMERODOCUMENTO==df_info_clientes.CustomerId])
DF3.show()

+---------------+----------+----------+---------------+----+----+------------+---------+--------------------+------------------+-----------+----------+---+--------+----------+--------+-----------------+------+----------+
|NUMERODOCUMENTO|    APEPAT|    APEMAT|        NOMBRES|EDAD|SEXO|DEPARTAMENTO|PROVINCIA|            DISTRITO|ESTADOCIVILTITULAR|IncomeLevel|CustomerId|TOB|   Bal01|MaxDqBin01|MtgBal01|NonBankTradesDq01|FlagGB|FlagSample|
+---------------+----------+----------+---------------+----+----+------------+---------+--------------------+------------------+-----------+----------+---+--------+----------+--------+-----------------+------+----------+
|       41011903|      DIAZ|   CORDOVA|  LORENA ROXANA|  33|   F|        LIMA|     LIMA|   MAGDALENA DEL MAR|                CS|          2|  41011903| 43|  613.21|         0|       0|                0|     1|         1|
|       41011903|      DIAZ|   CORDOVA|  LORENA ROXANA|  33|   F|        LIMA|     LIMA|   MAGDALENA DEL MAR|       

In [7]:
DF3.count()

15404

In [8]:
train = DF3.filter("FlagSample == 1")
train.show()

+---------------+----------+----------+---------------+----+----+------------+---------+--------------------+------------------+-----------+----------+---+--------+----------+--------+-----------------+------+----------+
|NUMERODOCUMENTO|    APEPAT|    APEMAT|        NOMBRES|EDAD|SEXO|DEPARTAMENTO|PROVINCIA|            DISTRITO|ESTADOCIVILTITULAR|IncomeLevel|CustomerId|TOB|   Bal01|MaxDqBin01|MtgBal01|NonBankTradesDq01|FlagGB|FlagSample|
+---------------+----------+----------+---------------+----+----+------------+---------+--------------------+------------------+-----------+----------+---+--------+----------+--------+-----------------+------+----------+
|       41011903|      DIAZ|   CORDOVA|  LORENA ROXANA|  33|   F|        LIMA|     LIMA|   MAGDALENA DEL MAR|                CS|          2|  41011903| 43|  613.21|         0|       0|                0|     1|         1|
|       41011903|      DIAZ|   CORDOVA|  LORENA ROXANA|  33|   F|        LIMA|     LIMA|   MAGDALENA DEL MAR|       

In [9]:
test = DF3.filter("FlagSample == 0")
test.show()

+---------------+-----------+----------+--------------------+----+----+------------+---------+--------------------+------------------+-----------+----------+---+-------+----------+--------+-----------------+------+----------+
|NUMERODOCUMENTO|     APEPAT|    APEMAT|             NOMBRES|EDAD|SEXO|DEPARTAMENTO|PROVINCIA|            DISTRITO|ESTADOCIVILTITULAR|IncomeLevel|CustomerId|TOB|  Bal01|MaxDqBin01|MtgBal01|NonBankTradesDq01|FlagGB|FlagSample|
+---------------+-----------+----------+--------------------+----+----+------------+---------+--------------------+------------------+-----------+----------+---+-------+----------+--------+-----------------+------+----------+
|       41143790|     LIZANA|  CASTILLO|          KELY ROCIO|  33|   F|        LIMA|     LIMA|                 ATE|                CS|          0|  41143790|  6| 798.64|         0|       0|                0|     0|         0|
|       41143790|     LIZANA|  CASTILLO|          KELY ROCIO|  33|   F|        LIMA|     LIMA|  

In [10]:
train.dtypes

[('NUMERODOCUMENTO', 'string'),
 ('APEPAT', 'string'),
 ('APEMAT', 'string'),
 ('NOMBRES', 'string'),
 ('EDAD', 'int'),
 ('SEXO', 'string'),
 ('DEPARTAMENTO', 'string'),
 ('PROVINCIA', 'string'),
 ('DISTRITO', 'string'),
 ('ESTADOCIVILTITULAR', 'string'),
 ('IncomeLevel', 'int'),
 ('CustomerId', 'string'),
 ('TOB', 'string'),
 ('Bal01', 'string'),
 ('MaxDqBin01', 'string'),
 ('MtgBal01', 'string'),
 ('NonBankTradesDq01', 'string'),
 ('FlagGB', 'string'),
 ('FlagSample', 'string')]

In [11]:
test.dtypes

[('NUMERODOCUMENTO', 'string'),
 ('APEPAT', 'string'),
 ('APEMAT', 'string'),
 ('NOMBRES', 'string'),
 ('EDAD', 'int'),
 ('SEXO', 'string'),
 ('DEPARTAMENTO', 'string'),
 ('PROVINCIA', 'string'),
 ('DISTRITO', 'string'),
 ('ESTADOCIVILTITULAR', 'string'),
 ('IncomeLevel', 'int'),
 ('CustomerId', 'string'),
 ('TOB', 'string'),
 ('Bal01', 'string'),
 ('MaxDqBin01', 'string'),
 ('MtgBal01', 'string'),
 ('NonBankTradesDq01', 'string'),
 ('FlagGB', 'string'),
 ('FlagSample', 'string')]

In [12]:
train.show()

+---------------+----------+----------+---------------+----+----+------------+---------+--------------------+------------------+-----------+----------+---+--------+----------+--------+-----------------+------+----------+
|NUMERODOCUMENTO|    APEPAT|    APEMAT|        NOMBRES|EDAD|SEXO|DEPARTAMENTO|PROVINCIA|            DISTRITO|ESTADOCIVILTITULAR|IncomeLevel|CustomerId|TOB|   Bal01|MaxDqBin01|MtgBal01|NonBankTradesDq01|FlagGB|FlagSample|
+---------------+----------+----------+---------------+----+----+------------+---------+--------------------+------------------+-----------+----------+---+--------+----------+--------+-----------------+------+----------+
|       41011903|      DIAZ|   CORDOVA|  LORENA ROXANA|  33|   F|        LIMA|     LIMA|   MAGDALENA DEL MAR|                CS|          2|  41011903| 43|  613.21|         0|       0|                0|     1|         1|
|       41011903|      DIAZ|   CORDOVA|  LORENA ROXANA|  33|   F|        LIMA|     LIMA|   MAGDALENA DEL MAR|       

In [13]:
train.columns

['NUMERODOCUMENTO',
 'APEPAT',
 'APEMAT',
 'NOMBRES',
 'EDAD',
 'SEXO',
 'DEPARTAMENTO',
 'PROVINCIA',
 'DISTRITO',
 'ESTADOCIVILTITULAR',
 'IncomeLevel',
 'CustomerId',
 'TOB',
 'Bal01',
 'MaxDqBin01',
 'MtgBal01',
 'NonBankTradesDq01',
 'FlagGB',
 'FlagSample']

In [14]:
train_model = train[[
 'EDAD',
 'SEXO',
 'DEPARTAMENTO',
 'PROVINCIA',
 'DISTRITO',
 'ESTADOCIVILTITULAR',
 'IncomeLevel',
 'TOB',
 'Bal01',
 'MaxDqBin01',
 'MtgBal01',
 'NonBankTradesDq01',
 'FlagGB']]

In [15]:
test_model = test[[
 'EDAD',
 'SEXO',
 'DEPARTAMENTO',
 'PROVINCIA',
 'DISTRITO',
 'ESTADOCIVILTITULAR',
 'IncomeLevel',
 'TOB',
 'Bal01',
 'MaxDqBin01',
 'MtgBal01',
 'NonBankTradesDq01',
 'FlagGB']]

In [16]:
train_model.count(), test_model.count()

(11600, 3804)

In [17]:
train_model.dtypes

[('EDAD', 'int'),
 ('SEXO', 'string'),
 ('DEPARTAMENTO', 'string'),
 ('PROVINCIA', 'string'),
 ('DISTRITO', 'string'),
 ('ESTADOCIVILTITULAR', 'string'),
 ('IncomeLevel', 'int'),
 ('TOB', 'string'),
 ('Bal01', 'string'),
 ('MaxDqBin01', 'string'),
 ('MtgBal01', 'string'),
 ('NonBankTradesDq01', 'string'),
 ('FlagGB', 'string')]

In [18]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pandas.tools.plotting import scatter_matrix
%matplotlib inline 
import matplotlib.pyplot as plt, numpy as np

from pyspark.sql.functions import col

In [21]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer
list_category = ['ESTADOCIVILTITULAR', 'DISTRITO', 'PROVINCIA', 'DEPARTAMENTO', 'SEXO']
indexers = [StringIndexer(inputCol=column, outputCol=column+"_index").fit(train) for column in list_category ]
pipeline = Pipeline(stages=indexers)
train = pipeline.fit(train).transform(train)
train.dtypes

[('NUMERODOCUMENTO', 'string'),
 ('APEPAT', 'string'),
 ('APEMAT', 'string'),
 ('NOMBRES', 'string'),
 ('EDAD', 'int'),
 ('SEXO', 'string'),
 ('DEPARTAMENTO', 'string'),
 ('PROVINCIA', 'string'),
 ('DISTRITO', 'string'),
 ('ESTADOCIVILTITULAR', 'string'),
 ('IncomeLevel', 'int'),
 ('CustomerId', 'string'),
 ('TOB', 'string'),
 ('Bal01', 'string'),
 ('MaxDqBin01', 'string'),
 ('MtgBal01', 'string'),
 ('NonBankTradesDq01', 'string'),
 ('FlagGB', 'string'),
 ('FlagSample', 'string'),
 ('ESTADOCIVILTITULAR_index', 'double'),
 ('DISTRITO_index', 'double'),
 ('PROVINCIA_index', 'double'),
 ('DEPARTAMENTO_index', 'double'),
 ('SEXO_index', 'double')]

In [25]:
from pyspark.sql.types import DoubleType, IntegerType
list_val_col = []
list_val_numeric = ['FlagGB', 'NonBankTradesDq01', 'MtgBal01', 'MaxDqBin01', 'Bal01', 'TOB']
list_int = ['EDAD', 'IncomeLevel', 'DISTRITO_index', 'ESTADOCIVILTITULAR_index', 'PROVINCIA_index', 'DEPARTAMENTO_index', 'SEXO_index']

cols_select = [col(_).cast(IntegerType()) for _ in list_val_numeric] + [col(_) for _ in list_int] 
train_x = train.select(*cols_select)
train_x.show()

+------+-----------------+--------+----------+-----+---+----+-----------+--------------+------------------------+---------------+------------------+----------+
|FlagGB|NonBankTradesDq01|MtgBal01|MaxDqBin01|Bal01|TOB|EDAD|IncomeLevel|DISTRITO_index|ESTADOCIVILTITULAR_index|PROVINCIA_index|DEPARTAMENTO_index|SEXO_index|
+------+-----------------+--------+----------+-----+---+----+-----------+--------------+------------------------+---------------+------------------+----------+
|     1|                0|       0|         0|  613| 43|  33|          2|          29.0|                     1.0|            0.0|               0.0|       1.0|
|     1|                0|       0|         0|  613| 43|  33|          2|          29.0|                     1.0|            0.0|               0.0|       1.0|
|     1|                0|       0|         0|  568| 48|  35|          1|           9.0|                     1.0|            0.0|               0.0|       0.0|
|     1|                0|       0|     

In [26]:
list_category = ['ESTADOCIVILTITULAR', 'DISTRITO', 'PROVINCIA', 'DEPARTAMENTO', 'SEXO']
indexers = [StringIndexer(inputCol=column, outputCol=column+"_index").fit(test) for column in list_category ]
pipeline = Pipeline(stages=indexers)
test = pipeline.fit(test).transform(test)
test.dtypes

[('NUMERODOCUMENTO', 'string'),
 ('APEPAT', 'string'),
 ('APEMAT', 'string'),
 ('NOMBRES', 'string'),
 ('EDAD', 'int'),
 ('SEXO', 'string'),
 ('DEPARTAMENTO', 'string'),
 ('PROVINCIA', 'string'),
 ('DISTRITO', 'string'),
 ('ESTADOCIVILTITULAR', 'string'),
 ('IncomeLevel', 'int'),
 ('CustomerId', 'string'),
 ('TOB', 'string'),
 ('Bal01', 'string'),
 ('MaxDqBin01', 'string'),
 ('MtgBal01', 'string'),
 ('NonBankTradesDq01', 'string'),
 ('FlagGB', 'string'),
 ('FlagSample', 'string'),
 ('ESTADOCIVILTITULAR_index', 'double'),
 ('DISTRITO_index', 'double'),
 ('PROVINCIA_index', 'double'),
 ('DEPARTAMENTO_index', 'double'),
 ('SEXO_index', 'double')]

In [27]:
from pyspark.sql.types import DoubleType, IntegerType
list_val_col = []
list_val_numeric = ['FlagGB', 'NonBankTradesDq01', 'MtgBal01', 'MaxDqBin01', 'Bal01', 'TOB']
list_int = ['EDAD', 'IncomeLevel', 'DISTRITO_index', 'ESTADOCIVILTITULAR_index', 'PROVINCIA_index', 'DEPARTAMENTO_index', 'SEXO_index']

cols_select = [col(_).cast(IntegerType()) for _ in list_val_numeric] + [col(_) for _ in list_int] 
test_x = test.select(*cols_select)
test_x.show()

+------+-----------------+--------+----------+-----+---+----+-----------+--------------+------------------------+---------------+------------------+----------+
|FlagGB|NonBankTradesDq01|MtgBal01|MaxDqBin01|Bal01|TOB|EDAD|IncomeLevel|DISTRITO_index|ESTADOCIVILTITULAR_index|PROVINCIA_index|DEPARTAMENTO_index|SEXO_index|
+------+-----------------+--------+----------+-----+---+----+-----------+--------------+------------------------+---------------+------------------+----------+
|     0|                0|       0|         0|  798|  6|  33|          0|           3.0|                     1.0|            0.0|               0.0|       1.0|
|     0|                0|       0|         0|  798|  6|  33|          0|           3.0|                     1.0|            0.0|               0.0|       1.0|
|     1|                0|       0|         0| 4499| 30|  32|          1|          16.0|                     1.0|            0.0|               0.0|       1.0|
|     1|                0|       0|     

In [28]:
test_x.dtypes

[('FlagGB', 'int'),
 ('NonBankTradesDq01', 'int'),
 ('MtgBal01', 'int'),
 ('MaxDqBin01', 'int'),
 ('Bal01', 'int'),
 ('TOB', 'int'),
 ('EDAD', 'int'),
 ('IncomeLevel', 'int'),
 ('DISTRITO_index', 'double'),
 ('ESTADOCIVILTITULAR_index', 'double'),
 ('PROVINCIA_index', 'double'),
 ('DEPARTAMENTO_index', 'double'),
 ('SEXO_index', 'double')]

In [34]:
list(set(train_x.columns) - set('FlagGB'))


['EDAD',
 'SEXO_index',
 'NonBankTradesDq01',
 'Bal01',
 'MtgBal01',
 'MaxDqBin01',
 'FlagGB',
 'PROVINCIA_index',
 'ESTADOCIVILTITULAR_index',
 'TOB',
 'DEPARTAMENTO_index',
 'IncomeLevel',
 'DISTRITO_index']

In [35]:
target_train = train_x['FlagGB']
target_test = test_x['FlagGB']
train_x = train_x[list(set(train_x.columns) - set('FlagGB'))]
test_x = test_x[list(set(train_x.columns) - set('FlagGB'))]

In [37]:
target_train.columns, target_test.columns

(Column<FlagGB['columns']>, Column<FlagGB['columns']>)

In [38]:
train_x.columns

['EDAD',
 'SEXO_index',
 'NonBankTradesDq01',
 'Bal01',
 'MtgBal01',
 'MaxDqBin01',
 'FlagGB',
 'PROVINCIA_index',
 'ESTADOCIVILTITULAR_index',
 'TOB',
 'DEPARTAMENTO_index',
 'IncomeLevel',
 'DISTRITO_index']